In [2]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [3]:
spark = (SparkSession
        .builder
        .master("local[3]")
        .appName("my_spark_app")
        .config("spark.sql.shuffle.partition",2)
        .config("spark.cache.enabled","false")
        .config("spark.stage.reuse.enabled","false")
        .getOrCreate()
        )
spark

In [4]:
employee_df = (spark.read
                # .format("csv")
                .option("header","true")
                .option("inferSchema","true")
                .csv("./employee_data.csv")
                # .csv("./employee_data.csv")
                )

# employee_df.collect()
# new_df.info()
# new_df
partation_df = employee_df.repartition(2)

count_df = (partation_df
            .withColumn("Current Employee Rating", col("Current Employee Rating").cast("int"))
            .filter(col("Current Employee Rating") > 3)
            .select("FirstName","LastName","Title","EmployeeType","DOB","Current Employee Rating")
            .groupBy("EmployeeType")
            .count()
            )

count_df.collect()

[Row(EmployeeType='Contract', count=235),
 Row(EmployeeType='Full-Time', count=229),
 Row(EmployeeType='Part-Time', count=225)]

In [5]:
count_df.show()

+------------+-----+
|EmployeeType|count|
+------------+-----+
|    Contract|  235|
|   Full-Time|  229|
|   Part-Time|  225|
+------------+-----+

